$$\huge{\text{Practice with Greenplum and MADlib}}$$

In a previous session we saw examples of how to do regression and classification in Greenplum as applied to the Abalone dataset.  Now let's practice applying those techniques to predicting insurance claims. We will use a [sample insurance claims dataset](https://www.kaggle.com/easonlai/sample-insurance-claim-prediction-dataset#insurance3r2.csv) from Kaggle. 

The Kaggle page doesn't provide a whole lot of context, but as you might guess from the name of the competition and the column names below, the goal is to predict whether a person will file a claim (`insuranceclaim`). The predictors that are available are the other colums: age, sex, BMI, steps, children, whether they smoke, the region they are in and "charges". The objective is to predict a claim.

| Column # | Column name |
|  ------  |  ---------  |
|    1     | age |
|    2     | sex |
|    3     | bmi |
|    4     | steps |
|    5     | children |
|    6     | smoker |
|    7     | region |
|    8     | charges |
|    9     | insuranceclaim |

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preliminaries</a></span><ul class="toc-item"><li><span><a href="#Connect-to-the-database" data-toc-modified-id="Connect-to-the-database-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Connect to the database</a></span></li><li><span><a href="#Imports-and-globals" data-toc-modified-id="Imports-and-globals-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Imports and globals</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Load Data</a></span></li></ul></li><li><span><a href="#Explore-data" data-toc-modified-id="Explore-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Explore data</a></span></li><li><span><a href="#Encode-categorical-variables" data-toc-modified-id="Encode-categorical-variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Encode categorical variables</a></span></li><li><span><a href="#Train-Test-split" data-toc-modified-id="Train-Test-split-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train-Test split</a></span></li><li><span><a href="#Modeling-(Classification)" data-toc-modified-id="Modeling-(Classification)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modeling (Classification)</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Random-Forest-Classifier" data-toc-modified-id="Random-Forest-Classifier-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Random Forest Classifier</a></span></li></ul></li></ul></div>

# Preliminaries

## Connect to the database

`dbconnect` is a custom module that helps with connecting to the database and loading an extension called `sql_magic`, which enables the ability to run a query by putting `%%read_sql` at the top of a cell. 

In [ ]:
%matplotlib inline

In [ ]:
import dbconnect

The credential file contents should look something like below. 

    [database_creds]
    host: <HOSTNAME_OR_IP>
    port: 5432
    user: <USERNAME>
    database: <DATABASE_NAME>
    password: <PASSWORD>

The values in angle brackets (\<...\>) are placeholders that need to be filled in. For example:

    [database_creds]
    host: 1.2.3.4
    port: 5432
    user: scott
    database: practice_db
    password: my_$ecretP@ss

Running the `connect_and_register_sql_magic()` function below will add a global variable `conn` that is a SQLAlchemy connection object.

In [ ]:
db_credential_file = '../.dbcred'
dbconnect.connect_and_register_sql_magic(
    db_credential_file,
    conn_name='conn'
)

## Imports and globals

In [ ]:
import math
import six
import pandas as pd
from sqlalchemy import create_engine

graphviz_installed = True

try:
    import graphviz
except ImportError:
    print("installing graphviz")
    install_graphviz = six.moves.input('Install `graphviz`? (y/n)')
    if install_graphviz == 'y':
        !pip install graphviz
    else:
        raise ImportError
except:
    graphviz_installed = False
    print("Could not load or install graphviz. Will not show random forest visualization below. ")

In [ ]:
pd.set_option('display.max_columns', 200)

In [ ]:
schema = 'ds_practice'

In [ ]:
%read_sql DROP SCHEMA IF EXISTS {schema} CASCADE;
%read_sql CREATE SCHEMA {schema};

## Load Data

In [ ]:
claims_data_path = '../input/insurance3r2.csv'

In [ ]:
df_claims = pd.read_csv(claims_data_path)

In [ ]:
df_claims.info()

**Add unique ID column**

In general, having a unique identifier for each record can be helpful. In particular, some algorithms use a unique ID internally to keep track of information. The parameter `index=True` in the Python code below uploads an index to the Greenplum table, matching the index of the local Pandas dataframe. In this case it is simply zero-indexed based on the order the data was read in from the CSV. The `index_label` parameter specifies the column name that the index will have in the database table (in this case, `id`). 

In [ ]:
df_claims.to_sql(
    'claims', 
    conn, 
    schema=schema, 
    if_exists='replace', 
    index=True, 
    index_label='id',
    chunksize=10000)

# Explore data

In [ ]:
%%read_sql
SELECT * FROM {schema}.claims
LIMIT 4

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_summary CASCADE;
SELECT madlib.summary(
    '{schema}.claims',   -- source_table
    '{schema}.claims_summary',  -- output_table
    NULL,  -- target_cols
    NULL,  -- grouping_cols
    TRUE,  -- get_distinct
    TRUE,  -- get_quartiles
    NULL,  -- ntile_array
    10,  -- how_many_mfv
    FALSE  -- get_estimates
)

In [ ]:
%%read_sql
SELECT * FROM {schema}.claims_summary
ORDER BY column_number

# Encode categorical variables

Convert region to `varchar` data type so the categorical encoding will work

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_region_text CASCADE;
CREATE TABLE {schema}.claims_region_text
AS
SELECT
    id,
    age,
    sex,
    bmi,
    steps,
    children,
    smoker,
    region::varchar as region,
    charges,
    insuranceclaim,
    insuranceclaim > 0 as insuranceclaim_bool
FROM {schema}.claims

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_encoded CASCADE;
SELECT madlib.encode_categorical_variables(
    '{schema}.claims_region_text',  -- input table
    '{schema}.claims_encoded',  -- output table
    'region'   -- categorical_cols
)

In [ ]:
%%read_sql
SELECT * FROM {schema}.claims_encoded
LIMIT 5

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_correlations CASCADE;
DROP TABLE IF EXISTS {schema}.claims_correlations_summary CASCADE;
SELECT
madlib.correlation(
    '{schema}.claims_encoded', -- source_table,
    '{schema}.claims_correlations', -- output_table,
    'age,sex,bmi,steps,children,smoker,charges,insuranceclaim,region_0,region_1,region_2,region_3', -- target_cols,
    TRUE, -- verbose,
    NULL  -- grouping_columns
)

In [ ]:
%%read_sql
SELECT * FROM {schema}.claims_correlations
ORDER BY
    column_position

# Train-Test split

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_eval CASCADE;
DROP TABLE IF EXISTS {schema}.claims_eval_train CASCADE;
DROP TABLE IF EXISTS {schema}.claims_eval_test CASCADE;
SELECT madlib.train_test_split(
    '{schema}.claims_encoded', -- source_table,
    '{schema}.claims_eval', -- output_table,
    0.7, -- train_proportion,
    NULL, -- test_proportion,
    NULL, -- grouping_cols,
    'id,age,sex,bmi,steps,children,smoker,charges,region_0,region_1,region_2,region_3,insuranceclaim,insuranceclaim_bool', -- target_cols,
    FALSE, -- with_replacement,
    TRUE -- separate_output_tables
)

In [ ]:
%%read_sql
SELECT count(*) as n
FROM {schema}.claims_eval_train

In [ ]:
%%read_sql
SELECT count(*) as n
FROM {schema}.claims_eval_test

# Modeling (Classification)

Since the original target variable is binary, we'll stick to classification approaches. 

## Logistic Regression

Note: drop one of the 1-hot-encoded variables (region) to remove perfect collinearity

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_logreg_model;
DROP TABLE IF EXISTS {schema}.claims_logreg_model_summary;
SELECT madlib.logregr_train(
    '{schema}.claims_eval_train', -- source_table,
    '{schema}.claims_logreg_model', -- out_table,
    'insuranceclaim_bool', -- dependent_varname,
    'ARRAY[
        1,
        age,
        sex,
        bmi,
        steps,
        children,
        smoker,
        charges,
        region_0,
        region_1,
        region_2
    ]' -- independent_varname,
    --, -- grouping_cols,
    --, -- max_iter,
    --, -- optimizer,
    --, -- tolerance,
     -- verbose
)

In [ ]:
%%read_sql
SELECT * FROM {schema}.claims_logreg_model

In [ ]:
%%read_sql
SELECT unnest(coef) as coef
FROM {schema}.claims_logreg_model

`f_unnest_ord(anyarray)` is a PL/SQL user-defined function for unnesting an array and getting the 1-indexed index for each value. The benefit of using this function is that there aren't guarantees that simply using `unnest()` will maintain the original order with the array in the results that you get back. 

`f_unnest_ord()` was taken from this [StackOverflow answer](https://stackoverflow.com/a/8767450/1789708)

In [ ]:
%%read_sql
CREATE OR REPLACE FUNCTION 
    {schema}.f_unnest_ord(anyarray, OUT val anyelement, OUT ordinality integer)
  RETURNS SETOF record LANGUAGE sql IMMUTABLE AS
'SELECT $1[i], i - array_lower($1,1) + 1
 FROM   generate_series(array_lower($1,1), array_upper($1,1)) i';

In [ ]:
%%read_sql
SELECT ({schema}.f_unnest_ord(coef)).*
FROM {schema}.claims_logreg_model

The `%%read_sql` magic command can optionally store the query result in a Pandas DataFrame if you supply a name to store it in. In the example below, `logreg_coefs` is the variable name that we will store a 1-by-1 shaped dataframe with a list of the coefficients. 

In [ ]:
%%read_sql logreg_coefs
SELECT coef 
FROM {schema}.claims_logreg_model

So we can view the coefficients with the variable names beside them, we have copied the names from the modeling step and `zip` them together with the coefficient values

In [ ]:
logreg_coef_names = (
    'intercept',
    'age',
    'sex',
    'bmi',
    'steps',
    'children',
    'smoker',
    'charges',
    'region_0',
    'region_1',
    'region_2'
)
tuple(zip(logreg_coef_names, logreg_coefs.iloc[0,0]))

**Cross join model with table to be scored**

Now that we have a model with coefficients, we can make predictions on records previously unseen by the model. In the current version of MADlib (1.15.1), the way to predict probability using a logistic regression model is to `CROSS JOIN` the test set records with the single-row model table. A `CROSS JOIN` produces the cartesian product between all records in both tables, meaning it pairs every record from one table with every record in the other table. In Postgres/Greenplum this can be done by explicitly using the `CROSS JOIN` statement, or you can simply list the two tables in the `FROM` clause separated by a comma. 

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_logreg_test_proba CASCADE;
CREATE TABLE {schema}.claims_logreg_test_proba
AS
SELECT madlib.logregr_predict_prob(
        coef, 
        ARRAY[
            1,
            age,
            sex,
            bmi,
            steps,
            children,
            smoker,
            charges,
            region_0,
            region_1,
            region_2
        ] 
    ) as proba,
    test.insuranceclaim
FROM {schema}.claims_eval_test test, {schema}.claims_logreg_model model
;

In [ ]:
%%read_sql
SELECT * FROM {schema}.claims_logreg_test_proba
LIMIT 6

In [ ]:
%%read_sql
SELECT madlib.area_under_roc(
    '{schema}.claims_logreg_test_proba', -- table_in, 
    '{schema}.claims_logreg_test_auc',  --table_out,
    'proba',  -- prediction_col, 
    'insuranceclaim'  --observed_col, 
)

In [ ]:
%%read_sql
SELECT *
FROM {schema}.claims_logreg_test_auc

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_logreg_test_predict;
CREATE TABLE {schema}.claims_logreg_test_predict
AS
SELECT
    (proba >= 0.5)::integer as predicted,
    insuranceclaim::integer as actual
FROM {schema}.claims_logreg_test_proba

In [ ]:
%%read_sql
SELECT
madlib.confusion_matrix(
    '{schema}.claims_logreg_test_predict', -- table_in
    '{schema}.claims_logreg_test_conf_matrix', -- table_out
    'predicted',  --prediction_col
    'actual' --observation_col
)

In [ ]:
%%read_sql
SELECT 
    row_id,
    class,
    confusion_arr[1] as predicted_0,
    confusion_arr[2] as predicted_1
FROM {schema}.claims_logreg_test_conf_matrix
ORDER BY row_id, class

Get ROC values (thresholds, true-positives, false-positives)

In [ ]:
%%read_sql
SELECT
madlib.binary_classifier(
    '{schema}.claims_logreg_test_proba', -- table_in
    '{schema}.claims_logreg_test_binary_metrics', -- table_out
    'proba',  --prediction_col
    'insuranceclaim' --observation_col
)

The `-d` flag for the `%%read_sql` magic command below keeps it from displaying the query result, which in this case is many rows that we want stored in the `logreg_metrics` dataframe but don't want to print the whole thing. 

In [ ]:
%%read_sql -d logreg_metrics
SELECT *
FROM {schema}.claims_logreg_test_binary_metrics
ORDER BY threshold

In [ ]:
logreg_metrics.head()

In [ ]:
logreg_metrics.plot('fpr', 'tpr')

## Random Forest Classifier

In [ ]:
%%read_sql
SELECT * 
FROM {schema}.claims_eval_train
LIMIT 5

In [ ]:
%%read_sql
SELECT
madlib.forest_train(
    '{schema}.claims_eval_train',  -- training_table_name
    '{schema}.claims_rf_model',  -- output_table_name
    'id',  -- id_col_name
    'insuranceclaim_bool',  -- dependent_variable
    'age,sex,bmi,steps,children,smoker,charges,region_0,region_1,region_2,region_3',  -- list_of_features
    NULL,  -- list_of_features_to_exclude
    NULL,  -- grouping_columns
    10  -- number of trees
)


We can peek at the Random Forest model table, though its contents are encoded in a not so human-friendly format.

In [ ]:
%%read_sql
SELECT count(*) 
FROM {schema}.claims_rf_model

In [ ]:
%%read_sql
SELECT * 
FROM {schema}.claims_rf_model
LIMIT 5

In [ ]:
%%read_sql rf_tree1
SELECT madlib.get_tree(
    '{schema}.claims_rf_model',
    1,
    1,
    FALSE  -- return results in dot_format? (boolean)
) 

In [ ]:
print(rf_tree1.iloc[0,0])

In [ ]:
%%read_sql rf_tree1_dot
SELECT madlib.get_tree(
    '{schema}.claims_rf_model',
    1,
    1,
    TRUE  -- return results in dot_format? (boolean)
) 

In [ ]:
if graphviz_installed:
    rf_dot_source = graphviz.Source(rf_tree1_dot.iloc[0,0])
    display(rf_dot_source)

In [ ]:
%%read_sql -d
DROP TABLE IF EXISTS {schema}.claims_rf_test_proba;
SELECT
madlib.forest_predict(
    '{schema}.claims_rf_model',  -- random_forest_model
    '{schema}.claims_eval_test',  -- new_data_table
    '{schema}.claims_rf_test_proba',  -- output_table
    'prob'  -- type
)

In [ ]:
%%read_sql
SELECT * 
FROM {schema}.claims_rf_test_proba
LIMIT 5

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_rf_test_predict_actual;
CREATE TABLE {schema}.claims_rf_test_predict_actual
AS
SELECT 
    test.id,
    prob.estimated_prob_true,
    prob.estimated_prob_true >= 0.5 as predicted_class,
    test.insuranceclaim_bool as actual_class
FROM 
    {schema}.claims_rf_test_proba prob
INNER JOIN
    {schema}.claims_eval_test test
ON
    prob.id = test.id

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.claims_rf_test_binary_metrics;
SELECT
madlib.binary_classifier(
    '{schema}.claims_rf_test_predict_actual', -- table_in
    '{schema}.claims_rf_test_binary_metrics', -- table_out
    'estimated_prob_true',  --prediction_col
    'actual_class' --observation_col
)

In [ ]:
%%read_sql
SELECT * 
FROM {schema}.claims_rf_test_binary_metrics
ORDER BY threshold
LIMIT 5

In [ ]:
%%read_sql -d rf_metrics
SELECT fpr, tpr
FROM {schema}.claims_rf_test_binary_metrics
ORDER BY threshold

In [ ]:
rf_metrics.plot('fpr', 'tpr')

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.abalone_rf_test_auc CASCADE;
SELECT
madlib.area_under_roc(
    '{schema}.claims_rf_test_predict_actual', -- table_in
    '{schema}.claims_rf_test_auc', -- table_out
    'estimated_prob_true',  --prediction_col
    'actual_class' --observation_col
)

In [ ]:
%%read_sql
SELECT * 
FROM {schema}.claims_rf_test_auc
LIMIT 5